<a href="https://colab.research.google.com/github/Sabarinathan-369/Predicting-Pokemon-Legendary-Status/blob/main/TensforFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries**

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import *

**Load & Preprocess Data**

In [18]:
df=pd.read_csv("/content/all_pokemon_data.csv")
df.head()

,Name,National Dex #,Primary Typing,Secondary Typing,Secondary Typing Flag,Generation,Legendary Status,Form,Alt Form Flag,Evolution Stage,...,Weight (hg),Height (in),Weight (lbs),Base Stat Total,Health,Attack,Defense,Special Attack,Special Defense,Speed
0,bulbasaur,1,grass,poison,True,generation-i,False,Base,False,1,...,69,28,15,318,45,49,49,65,65,45
1,ivysaur,2,grass,poison,True,generation-i,False,Base,False,2,...,130,39,29,405,60,62,63,80,80,60
2,venusaur,3,grass,poison,True,generation-i,False,Base,False,3,...,1000,79,220,525,80,82,83,100,100,80
3,venusaur-mega,3,grass,poison,True,generation-i,True,Mega,True,3,...,1555,94,343,625,80,100,123,122,120,80
4,charmander,4,fire,NaN,False,generation-i,False,Base,False,1,...,85,24,19,309,39,52,43,60,50,65


In [19]:
df.dtypes

,0
Name,object
National Dex #,int64
Primary Typing,object
Secondary Typing,object
Secondary Typing Flag,bool
Generation,object
Legendary Status,bool
Form,object
Alt Form Flag,bool
Evolution Stage,int64


In [20]:
df.isnull().sum()

,0
Name,0
National Dex #,0
Primary Typing,0
Secondary Typing,530
Secondary Typing Flag,0
Generation,0
Legendary Status,0
Form,0
Alt Form Flag,0
Evolution Stage,0


seems like `secondary typing` has 530 Nan

In [21]:
df['Secondary Typing']=df['Secondary Typing'].fillna('None')
df['Legendary Status']=df['Legendary Status'].map({False:0,True:1})
df = pd.get_dummies(df, columns=['Primary Typing', 'Secondary Typing', 'Generation', 'Color ID'])

In [22]:
num=['Health','Attack','Defense','Special Attack','Special Defense','Speed']
scaler=StandardScaler()
df[num]=scaler.fit_transform(df[num])

# drop irrelevant columns
df=df.drop(columns=['Name','National Dex #','Form','Alt Form Flag','Evolution Stage'])

**Split the data into train/test**

In [23]:
x=df.drop(columns=['Legendary Status'])
y=df['Legendary Status']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)


**Build Tensorflow model**

In [24]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(128,activation='relu',input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

**Train the model**

In [25]:
class_weights={0:1.,1:len(y_train[y_train==0])/len(y_train[y_train==1])}
history=model.fit(
    x_train,y_train,epochs=20,batch_size=32,validation_split=0.2,class_weight=class_weights
)

Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5933 - loss: 44.7134 - val_accuracy: 0.7211 - val_loss: 11.8556
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6123 - loss: 15.1935 - val_accuracy: 0.6947 - val_loss: 14.9928
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6073 - loss: 13.5247 - val_accuracy: 0.4895 - val_loss: 14.5460
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5028 - loss: 19.7989 - val_accuracy: 0.7842 - val_loss: 10.7413
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6921 - loss: 15.6944 - val_accuracy: 0.7842 - val_loss: 10.2024
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7094 - loss: 9.1858 - val_accuracy: 0.5053 - val_loss: 10.7720
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6979 - loss: 9.4422 - val_accuracy: 0.4895 - val_loss: 15.7248
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6610 - loss: 8.8035 - val_accuracy: 0

**Evaluate performace**

In [26]:
y_pred=(model.predict(x_test)>0.5).astype('int32')

print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       197
           1       0.61      0.55      0.58        40

    accuracy                           0.86       237
   macro avg       0.76      0.74      0.75       237
weighted avg       0.86      0.86      0.86       237

[[183  14]
 [ 18  22]]
